In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import tree
import gc

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./"))

# Any results you write to the current directory are saved as output.

['notebook (1).ipynb', '.DS_Store', 'Untitled.ipynb', 'test.csv', 'train_sample.csv', 'train.csv', '.ipynb_checkpoints']


In [2]:
path = './'

train_skip = skip = range(1, 140000000)
train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8'
        }

In [3]:
print("loading training data")
skip = range(1, 140000000)
train= pd.read_csv(path + "train.csv", skiprows=train_skip, dtype=dtypes, header=0, usecols=train_cols)
test = pd.read_csv(path + "test.csv", dtype=dtypes, header=0, usecols=test_cols)

loading training data


In [4]:
len_train = len(train)
train=train.append(test)
del test
gc.collect()

14

In [5]:
train['hour'] = pd.to_datetime(train.click_time).dt.hour.astype('uint8')
train['day'] = pd.to_datetime(train.click_time).dt.day.astype('uint8')

In [7]:
n_chans = train[['ip','day','hour','channel']].groupby(by=['ip','day',
          'hour'])[['channel']].count().reset_index().rename(columns={'channel': 'n_channels'})

In [8]:
train = train.merge(n_chans, on=['ip', 'day', 'hour'], how='left')

In [9]:
train = train.drop(['click_time', 'ip'], axis=1)

In [10]:
train['n_channels'] = train['n_channels'].astype('uint16')

In [11]:
x = train.loc[:len_train - 1, train.columns != 'is_attributed']
y = train.loc[:len_train - 1, train.columns == 'is_attributed']

In [17]:
x_test = train.loc[len_train:, train.columns != 'is_attributed']

In [18]:
x_test

,app,channel,device,os,hour,day,n_channels
44903891,9,107,1,3,4,10,34
44903892,9,466,1,3,4,10,403
44903893,21,128,1,19,4,10,229
44903894,15,111,1,13,4,10,239
44903895,12,328,1,13,4,10,60
44903896,18,107,1,13,4,10,120
44903897,3,137,1,1,4,10,90
44903898,27,153,1,19,4,10,93
44903899,18,107,1,10,4,10,106
44903900,12,424,1,53,4,10,539


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=1)

In [14]:
model = DecisionTreeClassifier(max_depth=5)

In [15]:
model.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [23]:
res = model.predict(x_test).astype(int)

In [26]:
submit = pd.read_csv(path + "test.csv", dtype='int', usecols=['click_id'])
submit['is_attributed'] = res

In [27]:
submit.to_csv('submission.csv', index=False)